In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

# Load data

In [43]:
import org.um.feri.ears.algorithms.gp.GPAlgorithmExecutor
import org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData
import org.um.feri.ears.util.gp_stats.GPProgramSolutionSimple

var dataFile = "C:\\Users\\marko\\UnityProjects\\GenIATraP_refactor\\GeneralTrainingEnvironmentForMAS\\evo_monitor\\src\\data\\multiConfigurationPrograssData.ser"
var progressData = GPAlgorithmMultiConfigurationsProgressData.deserializeState(dataFile)
System.out.println("Progress data loaded from file: " + progressData)

System.out.println("Data size: " +progressData.multiConfigurationProgressData.size)

Progress data loaded from file: org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData@12023a41
Data size: 1


# Prepare helper classes & functions

In [45]:
class SelectedIndividual(
    var selectedConfiguration: Int,
    var selectedRun: Int,
    var selectedGeneration: Int,
    var bestWorstIndividual: Int, // 0 = best, 1 = worst
) {
    constructor() : this(0, 0, 0, 0)
}

In [46]:
import org.um.feri.ears.individual.representations.gp.IndividualMatchResult

fun sumIndividualValues(individualMatchResult: IndividualMatchResult): Double {
    var sum = 0.0
    for (key in individualMatchResult.individualValues.keys) {
        sum += individualMatchResult.individualValues[key]!!
    }

    return sum
}

In [138]:
import org.yaml.snakeyaml.util.Tuple

// Prepare data (best, worst and average individual rating per generation)
fun prepareData(selectedConfiguration: Int, selectedRun: Int, bestGenIndividualsRating: MutableList<GPProgramSolutionSimple>?, worstGenIndividualsRating: MutableList<GPProgramSolutionSimple>?, avgGenIndividualRatings: MutableList<Double>?, bestGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>?, worstGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>?, avgGenIndividualAvgFitnesses: MutableList<Double>?){

    var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
    var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

    for (genProgressData in multiRunProgressData.gensProgressData) {
        var bestGenIndividualRating = genProgressData.population[0]
        var worstIndividualRating = genProgressData.population[0]
        var avgGenIndividualRating = 0.0

        var bestGenIndividualAvgFitness = genProgressData.population[0]
        var worstIndividualAvgFitness = genProgressData.population[0]
        var avgGenIndividualAvgFitness = 0.0

        for (solution in genProgressData.population) {
            if (solution.finalIndividualFitness.additionalValues["Rating"]!! < bestGenIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                bestGenIndividualRating = solution
            }

            if (solution.finalIndividualFitness.additionalValues["Rating"]!! > worstIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                worstIndividualRating = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) < sumIndividualValues(bestGenIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                bestGenIndividualAvgFitness = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) > sumIndividualValues(worstIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                worstIndividualAvgFitness = solution
            }

            avgGenIndividualRating += solution.finalIndividualFitness.additionalValues["Rating"]!!
            avgGenIndividualAvgFitness += sumIndividualValues(solution.finalIndividualFitness.avgMatchResult)
        }

        if (bestGenIndividualsRating != null) {
            bestGenIndividualsRating.add(bestGenIndividualRating)
        }
        if (worstGenIndividualsRating != null) {
            worstGenIndividualsRating.add(worstIndividualRating)
        }

        avgGenIndividualRating /= genProgressData.population.size
        if (avgGenIndividualRatings != null) {
            avgGenIndividualRatings.add(avgGenIndividualRating)
        }

        if (bestGenIndividualAvgFitnesses != null) {
            bestGenIndividualAvgFitnesses.add(bestGenIndividualAvgFitness)
        }
        if (worstGenIndividualAvgFitnesses != null) {
            worstGenIndividualAvgFitnesses.add(worstIndividualAvgFitness)
        }

        avgGenIndividualAvgFitness /= genProgressData.population.size
        if (avgGenIndividualAvgFitnesses != null) {
            avgGenIndividualAvgFitnesses.add(avgGenIndividualAvgFitness)
        }
    }
}

In [120]:
// TODO Move this to GPProgramSolutionSimple
fun calculateDiversity(solution1: GPProgramSolutionSimple, solution2: GPProgramSolutionSimple, w1: Double, w2: Double) : Double {
    var nodeCountDiversity = 0.0

    // Gather all different node count key
    val nodeCountKeys = mutableSetOf<String>()
    nodeCountKeys.addAll(solution1.nodeCounts.keys)
    nodeCountKeys.addAll(solution2.nodeCounts.keys)

    // Calculate node count diversity
    for (key in nodeCountKeys) {
        val nodeCount1 = solution1.nodeCounts[key] ?: 0
        val nodeCount2 = solution2.nodeCounts[key] ?: 0
        nodeCountDiversity += Math.abs(nodeCount1 - nodeCount2)
    }

    // Normalize node count diversity
    nodeCountDiversity /= (solution1.treeSize + solution2.treeSize)

    // Calculate avg fitness diversity
    var avgFitnessDiversity = 0.0

    // Gather all different avg fitness individual values key
    val avgFitnessKeys = mutableSetOf<String>()
    avgFitnessKeys.addAll(solution1.finalIndividualFitness.avgMatchResult.individualValues.keys)
    avgFitnessKeys.addAll(solution2.finalIndividualFitness.avgMatchResult.individualValues.keys)

    // Calculate avg fitness diversity
    for (key in avgFitnessKeys) {
        val avgFitness1 = solution1.finalIndividualFitness.avgMatchResult.individualValues[key] ?: 0.0
        val avgFitness2 = solution2.finalIndividualFitness.avgMatchResult.individualValues[key] ?: 0.0
        avgFitnessDiversity += Math.abs(avgFitness1 - avgFitness2)
    }

    // Normalize avg fitness diversity
    avgFitnessDiversity /= (Math.abs(sumIndividualValues(solution1.finalIndividualFitness.avgMatchResult)) + Math.abs(sumIndividualValues(solution2.finalIndividualFitness.avgMatchResult)))

    //System.out.println("Node count diversity: " + nodeCountDiversity)
    //System.out.println("Avg fitness diversity: " + avgFitnessDiversity)

    // Calculate final diversity
    return w1 * avgFitnessDiversity + w2 * nodeCountDiversity
}

# Visualize data (line plot) - Best, worst and average individual rating per generation

In [139]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

// Prepare data
val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
val avgGenIndividualRatings = mutableListOf<Double>()

val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

var lastPhaseName = "";
val generations = mutableListOf<String>()
for (genProgressData in multiRunProgressData.gensProgressData) {
    generations.add(genProgressData.generation.toString() + genProgressData.executionPhaseName.substring(genProgressData.executionPhaseName.length - 6, genProgressData.executionPhaseName.length))
}

// Display data
val dfRating = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + worstGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + avgGenIndividualRatings,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

val dfAvgFitness = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + worstGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + avgGenIndividualAvgFitnesses,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

// The plot variable stores a plot with the temperature values for each city
val linePlotRating =
    letsPlot(dfRating.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(2000,400) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Generation Rating")

val linePlotAvgFitness =
    letsPlot(dfAvgFitness.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(2000,800) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Generation Average fitness")

val plots = listOf(linePlotRating, null, linePlotAvgFitness, null)
gggrid(plots, ncol = 2)

<path d="M1.0193499853701076 111.0737301975832 L1.0193499853701076 111.0737301975832 L6.116099912220645 138.53503977460392 L11.212849839071183 134.48839011810338 L16.30959976592172 143.6270404181241 L21.40634969277226 153.66004412868057 L26.503099619622798 150.42892618247592 L31.599849546473333 156.70821256471004 L36.69659947332387 149.32948041578152 L41.79334940017441 156.843987862768 L46.89009932702494 156.1631250507811 L51.986849253875484 159.2704487458742 L57.08359918072602 147.20100862749157 L62.18034910757655 155.13848750067325 L67.2770990344271 146.81905567252815 L72.37384896127764 150.6743984167226 L77.47059888812818 157.10016767042458 L82.56734881497871 147.05149615134187 L87.66409874182925 155.45106399352258 L92.76084866867978 148.1451240374769 L97.85759859553033 155.2300773510367 L102.95434852238087 159.41682394756785 L108.0510984492314 155.89116605207323 L113.14784837608194 160.02652075153796 L118.24459830293247 152.93798834617263 L123.34134822978301 160.51174015824282 L128.43809815663354 154.63093961044305 L133.53484808348406 156.83558590762703 L138.6315980103346 156.84562147313568 L143.72834793718516 159.91426944277435 L148.82509786403568 154.3123150464825 L153.92184779088623 148.12679789935237 L159.01859771773675 145.24428498010576 L164.1153476445873 159.49808492510667 L169.21209757143785 159.45041320437755 L174.30884749828837 159.1718455091823 L179.40559742513892 157.6486523513681 L184.50234735198944 159.45594520312258 L189.59909727884 158.6154786582148 L194.69584720569054 159.3839066986594 L199.79259713254106 157.5034689426927 L204.8893470593916 147.5484961164481 L209.98609698624213 149.0182330877313 L215.08284691309268 148.44968841402303 L220.17959683994323 145.028084073447 L225.27634676679375 144.4939417489814 L230.3730966936443 148.93096487934048 L235.46984662049482 147.67246486684888 L240.56659654734537 147.89221145055566 L245.6633464741959 148.25511799373245 L250.76009640104644 147.31556926727492 L255.856846327897 148.0991973096405 L260.95359625474754 148.24236098186134 L266.05034618159806 147.10279894238823 L271.14709610844864 148.5927221399646 L276.24384603529916 148.5927221399646 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 <path d="M1.0193499853701076 52.18343531006536 L1.0193499853701076 52.18343531006536 L6.116099912220645 53.276398613800794 L11.212849839071183 55.863291173264216 L16.30959976592172 47.274276506946066 L21.40634969277226 42.7974396930014 L26.503099619622798 29.21165829847166 L31.599849546473333 24.023025888953242 L36.69659947332387 27.25253250129522 L41.79334940017441 24.424251733504278 L46.89009932702494 26.36212574489773 L51.986849253875484 30.650954425653758 L57.08359918072602 11.260660231118806 L62.18034910757655 29.9238493020094 L67.2770990344271 28.63061650545074 L72.37384896127764 27.442361738768724 L77.47059888812818 27.10254479303856 L82.56734881497871 25.595030582003112 L87.66409874182925 21.841714230771014 L92.76084866867978 28.459445697301547 L97.85759859553033 20.63663327730206 L102.95434852238087 25.674851012217715 L108.0510984492314 22.97251417827208 L113.14784837608194 28.475683413081057 L118.24459830293247 25.72617607976909 L123.34134822978301 17.808074053198446 L128.43809815663354 24.44097767601867 L133.53484808348406 23.646207668395462 L138.6315980103346 20.651289361225597 L143.72834793718516 22.691637152057094 L148.82509786403568 26.860569970079013 L153.92184779088623 21.50666445711376 L159.01859771773675 26.583513364473866 L164.1153476445873 26.679213230012344 L169.21209757143785 19.347644154948487 L174.30884749828837 8.263644219726828 L179.40559742513892 23.944044540427477 L184.50234735198944 24.644870813657434 L189.59909727884 19.605331339444298 L194.69584720569054 23.851585906368477 L199.79259713254106 25.38777369210728 L204.8893470593916 12.463804613751915 L209.98609698624213 20.72368020574309 L215.08284691309268 20.56772573561853 L220.17959683994323 22.261674616038498 L225.27634676679375 21.0685041672501

# Visualize data (radar plot) - Comparison of individuals from different generations, runs & configurations

In [141]:
// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 0, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 20, 0))

val maxValues = mutableMapOf<String, Double>()
maxValues["SectorExploration"] = 5.0
maxValues["PowerUp_Pickup_Health"] = 10.0
maxValues["PowerUp_Pickup_Ammo"] = 15.0
maxValues["PowerUp_Pickup_Shield"] = 5.0
maxValues["MissilesFired"] = 20.0
maxValues["MissilesFiredAccuracy"] = 50.0
maxValues["SurvivalBonus"] = 5.0
maxValues["OpponentTrackingBonus"] = 5.0
maxValues["OpponentDestroyedBonus"] = 500.0
maxValues["DamageTakenPenalty"] = 50.0
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualRatings = mutableListOf<Double>()

    val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

// Find all fitness keys
val fitnessKeys = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    for (key in individual.finalIndividualFitness.avgMatchResult.individualValues.keys) {
        if (key !in fitnessKeys) {
            fitnessKeys.add(key)
        }
    }
}

// Order fitnessKeys
fitnessKeys.sort()

// Build labels for visualization
val labels = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    labels.addAll(fitnessKeys)
}

val datasets = mutableListOf<MutableList<Double>>()

// Build datasets for visualization
for (individual in selectedGPAlgorithmSolutionsSimple) {
    val dataset = mutableListOf<Double>()
    for (key in fitnessKeys) {
        if(individual.finalIndividualFitness.avgMatchResult.individualValues[key] == null) {
            dataset.add(0.0)
            System.out.println(key + ": 0.0")
            continue
        }
        dataset.add(Math.abs(individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!) / maxValues[key]!!)
        System.out.println(key + ": " + individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!)
    }
    System.out.println("/////////////////////////////")
    datasets.add(dataset)
}

var fitnessKeysIndexed = mutableListOf<Int>()
for (i in 0 until fitnessKeys.size) {
    fitnessKeysIndexed.add(i + 1)
}

var individualValueIdsMapped = mutableListOf<Int>()
var individualMapped = mutableListOf<String>()
var index = 0
for(dataset in datasets) {
    individualValueIdsMapped.addAll(fitnessKeysIndexed)
    individualMapped.addAll(List(fitnessKeys.size) { "Individual " + (index + 1) })
    index++
}

// Build map for visualization
val selectedIndividualsData = mapOf(
    "IndividualValueLabels" to labels,
    "IndividualValueId" to individualValueIdsMapped,
    "Individual" to individualMapped,
    "IndividualValues" to datasets.map { it }.toMutableList().flatten()
)

// Map each fitnessKey to a map 1 -> "fitnessKey1", 2 -> "fitnessKey2", ...
var labelsIndexed = fitnessKeys.mapIndexed { index, s -> index to s }.toMap().mapKeys { it.key + 1 }

letsPlot(selectedIndividualsData) +
        geomArea(flat = true) {   // <-- flat. I.e., do not transform segments to curves
            x = "IndividualValueId"; y = "IndividualValues"
        } +
        geomPoint() { x = "IndividualValueId"; y = "IndividualValues"; color="Individual" } +
        scaleXDiscrete(labels = labelsIndexed) +
        coordPolar()

MissilesFired: 0.0
MissilesFiredAccuracy: 0.0
OpponentDestroyedBonus: 0.0
OpponentTrackingBonus: 0.0
PowerUp_Pickup_Ammo: 0.0
SectorExploration: -1.6667001
SurvivalBonus: -5.0
/////////////////////////////
MissilesFired: -5.0
MissilesFiredAccuracy: -12.5
OpponentDestroyedBonus: -125.0
OpponentTrackingBonus: -2.6062498
PowerUp_Pickup_Ammo: -1.875
SectorExploration: -2.8332999
SurvivalBonus: -5.0
/////////////////////////////


<path d="M170.5 100.04545454545456 L177.4057530686736 100.38471243900432 L184.24499996022723 101.39921888068147 L190.95187498838257 103.0792036188671 L197.46178728026769 105.40848748215933 L203.71204282183166 108.36463819363863 L209.642448235472 111.91918640595705 L215.19589047516592 116.03789987671627 L220.31888685632495 120.68111314367508 L224.96210012328373 125.80410952483408 L229.08081359404295 131.357551764528 L232.63536180636137 137.28795717816834 L235.59151251784067 143.53821271973231 L237.9207963811329 150.04812501161743 L239.60078111931853 156.7550000397728 L240.61528756099568 163.5942469313264 L240.95454545454544 170.5 L240.61528756099568 177.4057530686736 L239.60078111931853 184.2449999602272 L237.9207963811329 190.95187498838257 L235.59151251784067 197.46178728026769 L232.63536180636137 203.71204282183166 L229.08081359404298 209.64244823547196 L224.96210012328373 215.19589047516592 L220.31888685632495 220.31888685632492 L215.19589047516592 224.96210012328373 L209.64244823547196 229.08081359404298 L203.71204282183166 232.63536180636137 L197.46178728026769 235.59151251784067 L190.95187498838257 237.9207963811329 L184.24499996022723 239.60078111931853 L177.4057530686736 240.61528756099568 L170.5 240.95454545454544 L163.5942469313264 240.61528756099568 L156.75500003977277 239.60078111931853 L150.04812501161743 237.9207963811329 L143.53821271973231 235.59151251784067 L137.28795717816834 232.63536180636137 L131.35755176452804 229.08081359404298 L125.80410952483408 224.96210012328373 L120.68111314367506 220.31888685632495 L116.03789987671628 215.19589047516595 L111.91918640595703 209.642448235472 L108.36463819363863 203.71204282183166 L105.40848748215936 197.4617872802677 L103.07920361886711 190.95187498838257 L101.39921888068149 184.24499996022723 L100.3847124390043 177.40575306867356 L100.04545454545455 170.5 L100.3847124390043 163.59424693132644 L101.39921888068149 156.75500003977277 L103.0792036188671 150.04812501161746 L105.40848748215936 143.5382127197323 L108.36463819363863 137.28795717816834 L111.91918640595702 131.35755176452804 L116.03789987671627 125.80410952483408 L120.68111314367505 120.68111314367508 L125.80410952483405 116.03789987671627 L131.357551764528 111.91918640595705 L137.28795717816834 108.36463819363863 L143.53821271973226 105.40848748215936 L150.04812501161743 103.0792036188671 L156.75500003977277 101.3992188806815 L163.5942469313264 100.38471243900432 L170.5 100.04545454545456 " stroke="rgb(233,233,233)" stroke-opacity="1.0" stroke-width="1.0" fill="none">
 
 <path d="M170.5 29.590909090909122 L184.3115061373472 30.269424878008635 L197.98999992045444 32.29843776136295 L211.40374997676514 35.658407237734195 L224.42357456053537 40.31697496431866 L236.92408564366332 46.22927638727725 L248.78489647094398 53.3383728119141 L259.89178095033185 61.575799753432534 L270.1377737126499 70.86222628735015 L279.42420024656747 81.10821904966815 L287.6616271880859 92.21510352905605 L294.77072361272275 104.07591435633668 L300.68302503568134 116.57642543946463 L305.3415927622658 129.59625002323486 L308.70156223863705 143.01000007954556 L310.73057512199136 156.68849386265282 L311.4090909090909 170.5 L310.73057512199136 184.31150613734718 L308.70156223863705 197.98999992045444 L305.3415927622658 211.40374997676514 L300.68302503568134 224.42357456053537 L294.77072361272275 236.92408564366332 L287.66162718808596 248.78489647094392 L279.42420024656747 259.89178095033185 L270.1377737126499 270.13777371264985 L259.89178095033185 279.42420024656747 L248.78489647094392 287.66162718808596 L236.92408564366332 294.77072361272275 L224.42357456053537 300.68302503568134 L211.40374997676514 305.3415927622658 L197.98999992045447 308.70156223863705 L184.3115061373472 310.73057512199136 L170.5 311.4090909090909 L156.68849386265282 310.73057512199136 L143.01000007954553 308.70156223863705 L129.5962500232349 305.3415927622658 L116.57642543946463 300.68302503568134 L104.0759143563367 294.77072361272275 L92.2151035290561 287.661627188085

# Selected Individual Details

In [140]:
import org.jetbrains.letsPlot.intern.Plot

// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 0, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 20, 0))
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualRatings = mutableListOf<Double>()

    val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

var plots = mutableListOf<Plot>()

var index = 0
for (individual in selectedGPAlgorithmSolutionsSimple) {
    var nodeCountsData = mapOf(
        "NodeName" to individual.nodeCounts.keys.toList(),
        "NodeCount" to individual.nodeCounts.values.toList()
    )

    val blankTheme = theme(line=elementBlank(), axis=elementBlank())
    val (w, h) = 800 to 400
    val p = letsPlot(nodeCountsData) + ggsize(w,h) + blankTheme

    val plot = p + geomPie(stat = Stat.identity,
            size = 20, stroke = 1, color = "white", hole = 0.5) { slice = "NodeCount"; fill = "NodeName" } +
            blankTheme +
            scaleFillBrewer(palette = "Set1") + ggtitle("Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId)

    System.out.println(
        "Individual: " + "(Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId + ")\nChangesCount: " + individual.changesCount + "\nFunctionNodesCount: " + individual.functionNodes + "\nTerminalNodesCount: " + individual.terminalNodes + "\nTreeSize: " + individual.treeSize + "\nTreeDepth: " + individual.treeDepth + "\nFitness: " + (individual.finalIndividualFitness.additionalValues["Rating"].toString() + ", " + individual.finalIndividualFitness.additionalValues["StdDeviation"]) + "\nIndividualAvgFitness: " + sumIndividualValues(individual.finalIndividualFitness.avgMatchResult) + "\nIndividualAvgFitnessValues: " + individual.finalIndividualFitness.avgMatchResult.individualValues.map { it.key + ": " + it.value }.toString()
        + "\n\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"
    )

    index++

    plots.add(plot)
}

gggrid(plots, ncol = 2)

Individual: (Conf: 0 Run: 0 Gen: 0 IndividualID: 69)
ChangesCount: 1
FunctionNodesCount: 23
TerminalNodesCount: 37
TreeSize: 60
TreeDepth: 6
Fitness: -32.311954, 2.9166408
IndividualAvgFitness: -6.6667001
IndividualAvgFitnessValues: [SectorExploration: -1.6667001, SurvivalBonus: -5.0]
\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
Individual: (Conf: 0 Run: 0 Gen: 20 IndividualID: 78)
ChangesCount: 17
FunctionNodesCount: 82
TerminalNodesCount: 111
TreeSize: 193
TreeDepth: 12
Fitness: -45.332783, 3.4915457
IndividualAvgFitness: -154.8145497
IndividualAvgFitnessValues: [OpponentDestroyedBonus: -125.0, SectorExploration: -2.8332999, SurvivalBonus: -5.0, OpponentTrackingBonus: -2.6062498, PowerUp_Pickup_Ammo: -1.875, MissilesFired: -5.0, MissilesFiredAccuracy: -12.5]
\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\


Conf: 0 Run: 0 Gen: 0 IndividualID: 69 
 
 
 
 
 
 
 
 
 NodeName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Inverter 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RayHitObject 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Sequencer 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ShieldLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rotate 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 HealthLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AmmoLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MoveForward 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Shoot 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Selector 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Conf: 0 Run: 0 Gen: 20 IndividualID: 78 
 
 
 
 
 
 
 
 
 NodeName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RayHitObject 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Inverter 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Sequencer 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ShieldLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rotate 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 HealthLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AmmoLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MoveForward 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Shoot 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Selector

# Genetic Diversity Graph
Genetic diversity of the population based on the avg fitness and node counts

### Calculation of genetic diversity between 2 individuals
1. Difference in avg fitness -> df
2. Difference in node counts -> dn
3. Normalize both values and multiply by the weights -> diversity = w1 * df_norm +  w2 * dn_norm


In [149]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
val w1 = 0.5
val w2 = 0.5
// Input params for visualization END

val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()

prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, null, null, null, null, null)

val gpAlgorithmRunGeneticDiversity = mutableListOf<List<Double?>>()

for(i in 0 until bestGenIndividualsRating.size) {
    var gpAlgorithmGenGeneticDiversity = mutableListOf<Double?>()
    for (j in 0 until bestGenIndividualsRating.size) {
        if(j < i){
            var diversity = calculateDiversity(bestGenIndividualsRating[i], bestGenIndividualsRating[j], w1, w2)
            if(diversity > 1.0 || diversity < 0.0) {
                throw Exception("Diversity out of bounds: " + diversity)
            }
            gpAlgorithmGenGeneticDiversity.add(diversity)
        }
        else {
            gpAlgorithmGenGeneticDiversity.add(null)
        }
    }
    gpAlgorithmRunGeneticDiversity.add(gpAlgorithmGenGeneticDiversity)
}

var individuals = mutableListOf<Int>()
val generations = mutableListOf<String>()

for (i in 0 until gpAlgorithmRunGeneticDiversity.size) {
    generations.addAll(List(gpAlgorithmRunGeneticDiversity[i].size){(i + 1).toString()})

    for (i in 0 until gpAlgorithmRunGeneticDiversity[i].size) {
        individuals.add(i + 1)
    }
}

val df = dataFrameOf(
    "individuals" to individuals,
    "generations" to generations,
    "diversity" to gpAlgorithmRunGeneticDiversity.flatten()
)

val plot = letsPlot(df.toMap()) { x = "individuals"; y = "generations"; fill = "diversity" } + geomTile() + ggtitle("Genetic Diversity Graph")

plot

<rect fill="#808080" stroke="#ffffff" stroke-

# Master Tournament Graph
Tournament between generation best individuals

In [147]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

val n = 3
val Generation = mutableListOf<Int>()
val Rating = mutableListOf<Double>()

val masterTournamentGraphData = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].masterTournamentGraphData;
for (i in 0 until masterTournamentGraphData.size) {
    Generation.addAll(List(n) {i + 1})
    val rating = Math.abs(masterTournamentGraphData[i].finalIndividualFitness.additionalValues["Rating"]!!)
    Rating.addAll(listOf(rating - (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))
}

val data = mapOf<String, Any>(
    "Generation" to Generation,
    "Rating" to Rating
)

letsPlot(data) { x = "Generation"; y = "Rating"}  + geomBoxplot(whiskerWidth = 0.2) + ggtitle("Convergence graph (Tournament between generation best individuals)")

0 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 Convergence graph (Tournament between generation best individuals) 
 
 
 
 
 Rating 
 
 
 
 
 Generation

# Convergence graph
Best individual in the final generation is matched with every best individual from previous generations

In [148]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

val convergenceGraphData = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].convergenceGraphData;

var generations = mutableListOf<String>()
var fitnesses = mutableListOf<Float>()

for (individualMatch in convergenceGraphData.finalIndividualFitness.individualMatchResults){
    generations.add(individualMatch.opponentsIDs[0].toString())
    fitnesses.add(individualMatch.value)
}

// Display data
val df = dataFrameOf(
    "Generation" to generations,
    "Fitness" to fitnesses
)

val plot = letsPlot(df.toMap()) { x = "Generation"; y = "Fitness"} + geomLine() + geomPoint(shape = 20) + ggtitle("Convergence graph")

plot

<path d="M2.007929687702775 18.926606371792296 L2.007929687702775 18.926606371792296 L12.047578126216651 56.763611446551835 L22.087226564730525 303.58248049410616 L32.1268750032444 56.763611446551835 L42.16652344175828 303.58248049410616 L52.20617188027216 56.763611446551835 L62.24582031878603 56.763611446551835 L72.2854687572999 56.763611446551835 L82.32511719581377 56.763611446551835 L92.36476563432765 56.763611446551835 L102.40441407284153 50.06503185494754 L112.4440625113554 18.28134725649484 L122.48371094986928 14.456308594957434 L132.52335938838317 18.28134725649484 L142.56300782689704 50.06503185494754 L152.60265626541093 44.74090673089962 L162.6423047039248 19.06812092345139 L172.68195314243866 19.06812092345139 L182.72160158095255 44.74090673089962 L192.76125001946642 44.74090673089962 L202.8008984579803 283.96705544904444 L212.84054689649417 255.94274214914753 L222.88019533500804 45.39919647514773 L232.91984377352193 45.39919647514773 L242.9594922120358 17.74807564176793 L252.99914065054966 45.39919647514773 L263.0387890890635 17.74807564176793 L273.0784375275774 17.74807564176793 L283.11808596609126 17.74807564176793 L293.15773440460515 45.39919647514773 L303.19738284311904 17.74807564176793 L313.2370312816329 45.39919647514773 L323.2766797201468 45.39919647514773 L333.31632815866067 45.39919647514773 L343.3559765971745 45.39919647514773 L353.3956250356884 45.39919647514773 L363.4352734742023 17.74807564176793 L373.4749219127161 45.39919647514773 L383.51457035123 45.39919647514773 L393.5542187897439 45.39919647514773 L403.5938672282578 17.74807564176793 L413.63351566677164 45.39919647514773 L423.67316410528554 45.39919647514773 L433.71281254379943 17.74807564176793 L443.75246098231327 17.74807564176793 L453.79210942082716 17.74807564176793 L463.83175785934105 17.74807564176793 L473.8714062978549 45.39919647514773 L483.9110547363688 17.74807564176793 L493.9507031748827 45.39919647514773 L503.9903516133965 45.39919647514773 L514.0300000519105 45.39919647514773 L524.0696484904244 17.74807564176793 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 22 
 
 
 
 
 
 
 
 
 24 
 
 
 
 
 
 
 
 
 26 
 
 
 
 
 
 
 
 
 28 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 32 
 
 
 
 
 
 
 
 
 34 
 
 
 
 
 
 
 
 
 36 
 
 
 
 
 
 
 
 
 38 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 42 
 
 
 
 
 
 
 
 
 44 
 
 
 
 
 
 
 
 
 46 
 
 
 
 
 
 
 
 
 48 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 52 
 
 
 
 
 
 
 
 
 
 
 -300 
 
 
 
 
 
 
 -250 
 
 
 
 
 
 
 -200 
 
 
 
 
 
 
 -150 
 
 
 
 
 
 
 -100 
 
 
 
 
 
 
 -50 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 Convergence graph 
 
 
 
 
 Fitness 
 
 
 
 
 Generation